In [8]:
import numpy as np
import pandas as pd
from stat570.linear_model import linear_regression

np.set_printoptions(suppress=True)

In [4]:
mortality_data = pd.read_csv('mortality.txt')
mortality_data.head()

,log.s1,log.s3
1,-0.027825,-0.042655
2,-0.019877,-0.031257
3,-0.067820,-0.091980
4,-0.119681,-0.159399
5,-0.028417,-0.037638


In [29]:
estimates = linear_regression.LinearRegression.from_data_frame(mortality_data,['log.s3'], 'log.s1').coefficients_
estimates.index = ['$\\hat{\\beta}_0$', '$\\hat{\\beta}_1$']
with open('p2_estimates.tex', 'w') as f:
    f.write(estimates.to_latex(escape=False).replace('std_error', 'standard error'))
estimates

,estimate,std_error,t-statistic,p-value
$\hat{\beta}_0$,-0.002178,0.002584,-0.842783,4.038069e-01
$\hat{\beta}_1$,0.686650,0.030340,22.631990,3.179129e-26


## Prediction

In [33]:
X0 = np.log(0.95)
X0

-0.05129329438755058

In [32]:
1/estimates['estimate'][1]/5


0.29126917162426685